In [1]:
import pickle
from multiprocessing.connection import Listener
from threading import Thread

class RPCHandler:

    def __init__(self):
        self._functions = {}

    def register_function(self, func):
        self._functions[func.__name__] = func

    def handle_connection(self, conn):
        try:
            while True:
                func_name, args, kwargs = pickle.loads(conn.recv())
                try:
                    r = self._functions[func_name](*args, **kwargs)
                    conn.send(pickle.dumps(r))
                except Exception as e:
                    print(e)
                    print(type(e))
                    conn.send(pickle.dumps(e))

        except EOFError:
            pass

In [2]:
def rpc_server(handler, addr, authkey):
    sock = Listener(addr, authkey=authkey)
    while True:
        client = sock.accept()
        t = Thread(target=handler.handle_connection, args=(client, ))
        t.daemon = True
        t.start()


def add(x, y):
    return x + y


def sub(x, y):
    return x - y


handler = RPCHandler()
handler.register_function(add)
handler.register_function(sub)
rpc_server(handler, ('localhost', 25000), authkey=b'1234')

'fuck'
<class 'KeyError'>
'mul'
<class 'KeyError'>


KeyboardInterrupt: 